In [1]:
### libraries for the usage
import numpy as np 
import math 
import sys 
import pandas as pd 
import glob
from numpy import linalg as lng

from modules import Interaction,Files,Observable

### Number density calculations

In [4]:
def intham(mc_kagome,temp):
    """
    Initializing the interacting part of the hamiltonian given the 
    non-interacting hamiltonian
    """
    for j in range(10):
        noint_copy = mc_kagome.ham.copy()
        intmat = mc_kagome.intpart(mc_kagome.datadf, noint_copy, j)
        evlsi, evcsi = lng.eigh(intmat)
        observ = Observable(evlsi,evcsi,temp)
        be = observ.fermiFunc()
        observ.feCe(4)
        print(observ.n)

def main():
    """
    This is the main program this calls intham subroutine to initialize
    the interacting hamiltonian for different monte carlo configurations
    """
    # number of unit cell in x direction
    l_cell = int(12)  

    # total number of unit cells
    n_cell = int(l_cell**2)

    # number of sites in the unit cell
    s_cell = int(3)  

    # total number of site (for one spin type)
    n_sites = int(s_cell*n_cell)  
    
    # hopping strength
    hopping = 1.0   

    # dimensionality of the matrix
    dim_h = int(2*n_sites)

    # chemical potential
    mu = 13.5  

    # interaction strength
    u_int = 9.0  

    # temperature
    temp = 0.03

    ## file path were the files are stored
    file_path = './'
    ## pattern we want to search in the file name
    pattern = f'mc*L{l_cell}*Uint{u_int}*.dat'

    ## get the file names of files inside the folder
    file_dict = Files.getFiles(file_path, pattern)


    ### create the object
    mc_kagome = Interaction(l_cell, n_cell, s_cell, hopping, n_sites,
                            dim_h, mu, u_int)


    ### get data for each temperature into a dataframe
    datadf = mc_kagome.datadf(temp, file_dict)
    datadf['temp'] = temp
    mc_kagome.datadf = datadf
    ### set up the non-interacting hamiltonian
    mc_kagome.neighbour()  # set up the neighbour table

    ## initialize the non interacting hamiltonian
    ## ham is the non-interacting hamiltonian
    mc_kagome.haminit()
    intham(mc_kagome,temp)

   

In [5]:
if __name__=="__main__":
    main()
    

144 432


TypeError: 'float' object is not subscriptable